In [1]:
import base

import pandas as pd
import json
from collections import namedtuple
import re
import ngender


In [2]:
# my private modules
import fund_scanner.database
import fund_scanner.readurl

In [3]:
#page=page_index,items_in_one_page

url = 'http://fund.eastmoney.com/Data/Fund_JJJZ_Data.aspx?t=1&lx=1&letter=&gsid=&text=&sort=zdf,desc&page=1,100000&dt=1501051811136&atfc=&onlySale=0'

fund_scanner.readurl.read_from_url(url, 'tmp/bigfile')
body = fund_scanner.readurl.read_from_file('tmp/bigfile')


In [4]:
body = body[re.search('{', body).start():]
insider = re.search('\[\[(.*?)\]\]', body).group(1)
insider = '{"main":[['+insider+']]}'
obj = json.loads(insider, object_hook=lambda d: namedtuple('X', d.keys())(*d.values()))
df =pd.DataFrame(obj.main)

In [5]:
new_df = df.iloc[:,[0,1,3,4]]
new_df.columns = ['funds.funds_code', 'funds.funds_name_full', 
                  'funds_update.funds_price', 'funds_update.funds_price_adjust']
new_df.head(10)


,funds.funds_code,funds.funds_name_full,funds_update.funds_price,funds_update.funds_price_adjust
0,003620,招商睿乾混合,1.08,1.08
1,160221,国泰国证有色金属行业指数分级,1.1689,0.8012
2,165520,信诚中证800有色指数分级,1.2260,1.3160
3,257060,国联安商品ETF联接,0.6368,0.6368
4,161715,招商大宗商品(LOF),1.1480,1.2860
5,519173,浦银安盛睿智精选混合C,1.1080,1.1080
6,001559,天弘中证大宗商品股票指数C,0.8209,0.8209
7,519176,浦银安盛消费升级混合C,1.6330,1.6330
8,001558,天弘中证大宗商品股票指数A,0.8255,0.8255
9,519125,浦银安盛消费升级混合A,1.6420,1.6420


In [6]:
# Connect to the database
connection = fund_scanner.database.get_connection()

def write_to_sql(row):
    global connection
    with connection.cursor() as cursor:
        sql = 'INSERT INTO funds (funds_code,funds_name_full) VALUES (%s,%s) ON DUPLICATE KEY UPDATE funds_name_full=%s;'
        cursor.execute(sql, (row[0],row[1],row[1]))

    
try:
    new_df.apply(write_to_sql, axis=1)
    connection.commit()
finally:
    connection.close()

In [7]:
#new_df.head()

if 1==1:
    print('a')

if fund_scanner.database == database:
    print('y')
    
db


a


NameError: name 'database' is not defined